## Brief HowTo
- start the controller `ipcontroller --ip=<ip address>`
- copy the `ipcontroller-engine.json` to the correct folder on the engine machines (usually something like `/home/<user>/.ipython/profile_default/security/`)
- start the engines `ipcluster engines --n=<number of processes>`
- you can connect to the controller from a notebook on the same machine with `import ipyparallel as ipp; rc = ipp.Client()`
- executing `rc.ids` should show the ids of all connected engines

In [60]:
f = lambda x: x ** 4 - 3 * x

In [61]:
def integrate(f, a, b, n):
    dx = (b - a) / n
    s = 0.0
    x = a + 0.5 * dx
    for i in range(n):
        y = f(x)
        s += y
        x += dx
    return s * dx

In [151]:
ns = [16_000_000] * 8

In [1]:
import ipyparallel as ipp
rc = ipp.Client()

In [2]:
rc.ids

[0, 1, 2, 3]

In [158]:
dview = rc[:]
dview.push({'f': f, 'integrate': integrate}).wait()

True

In [152]:
%%time
serial_result = list(map(lambda n: integrate(f, -2, 2, n), ns))
serial_result

CPU times: user 27.1 s, sys: 0 ns, total: 27.1 s
Wall time: 26.4 s


[12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913]

In [159]:
%%time
dview = rc[:]
parallel_result = dview.map_sync(lambda n: integrate(f, -2, 2, n), ns)
parallel_result

CPU times: user 109 ms, sys: 126 ms, total: 235 ms
Wall time: 4.87 s


[12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913,
 12.800000002034913]